<a href="https://colab.research.google.com/github/Basifrank/GPT2_conversational_analyis/blob/master/conversational_analysis_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [4]:
#Cloning the gpt-2 model
!git clone https://github.com/tenoke/gpt-2

Cloning into 'gpt-2'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 4.39 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [5]:
#changing the root directory
cd gpt-2

/content/gpt-2


In [6]:
#Installing the required files in the requirements.txt file
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 604kB 10.1MB/s 
     |████████████████████████████████| 51kB 19.9MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=b18e55f055d2dedec584f5e3b1a8da75006b982db453e8241f377ff8c6f92d14
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533178 sha256=a9db5e0ce3e9f7dc2a7b1010345d8c9f90ac6889b82725886fc057760b181c32
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [7]:
#downloading the model
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 542kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 35.4Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 570kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:29, 48.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 4.93Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 32.9Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 34.0Mit/s]                                                       


In [9]:
#setting up the kaggle api so that we can access our ubuntu corpus dataset
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"basifrank","key":"f5f3c7a9c1f89b369d085833c612e5f3"}'}

In [10]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Oct 12 23:35 kaggle.json


In [0]:
#Installing the Kaggle API client
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d rtatman/ubuntu-dialogue-corpus

 99% 790M/799M [00:05<00:00, 143MB/s]
100% 799M/799M [00:05<00:00, 146MB/s]


In [14]:
!unzip ubuntu-dialogue-corpus.zip

Archive:  ubuntu-dialogue-corpus.zip
  inflating: Ubuntu-dialogue-corpus/dialogueText.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_196.csv  
  inflating: Ubuntu-dialogue-corpus/dialogueText_301.csv  
  inflating: toc.csv                 


In [0]:
!mkdir ubuntu-data ubuntu-npz

In [0]:
datasets = pd.read_csv('Ubuntu-dialogue-corpus/dialogueText_196.csv', parse_dates=['date'], chunksize=1200000)

In [0]:
i = 1

for dataset in datasets:
  dataset['date'] = dataset['date'].astype(int64) // 10**9
  
  text_corpus = ''
  current = None
  for msg in dataset.itertuples():
    if msg.dialogueID != current:
      current = msg.dialogueID
      text_corpus += '\n\n'
    try:
      text_corpus += f"({msg.date}) {msg._4}: {msg.text}\n"
    except KeyError:
      pass
  
  with open(f'ubuntu-data/ubuntu-cleaned-{i}.txt', 'w') as f:
    f.write(text_corpus)
  del(text_corpus)
  i +=1
    
del(datasets)
del(dataset)

In [18]:
!du -h ubuntu-data/*

91M	ubuntu-data/ubuntu-cleaned-1.txt
92M	ubuntu-data/ubuntu-cleaned-2.txt
90M	ubuntu-data/ubuntu-cleaned-3.txt
89M	ubuntu-data/ubuntu-cleaned-4.txt
89M	ubuntu-data/ubuntu-cleaned-5.txt
86M	ubuntu-data/ubuntu-cleaned-6.txt
85M	ubuntu-data/ubuntu-cleaned-7.txt
60M	ubuntu-data/ubuntu-cleaned-8.txt


In [20]:
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-1.txt ubuntu-npz/ubuntu-cleaned-1.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-2.txt ubuntu-npz/ubuntu-cleaned-2.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-3.txt ubuntu-npz/ubuntu-cleaned-3.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-4.txt ubuntu-npz/ubuntu-cleaned-4.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-5.txt ubuntu-npz/ubuntu-cleaned-5.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-6.txt ubuntu-npz/ubuntu-cleaned-6.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-7.txt ubuntu-npz/ubuntu-cleaned-7.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py ubuntu-data/ubuntu-cleaned-8.txt ubuntu-npz/ubuntu-cleaned-8.txt.npz --model_name 345M

Reading files
100% 1/1 [03:27<00:00, 207.89s/it]
Writing ubuntu-npz/ubuntu-cleaned-1.txt.npz
Reading files
100% 1/1 [03:33<00:00, 213.25s/it]
Writing ubuntu-npz/ubuntu-cleaned-2.txt.npz
Reading files
100% 1/1 [03:25<00:00, 205.22s/it]
Writing ubuntu-npz/ubuntu-cleaned-3.txt.npz
Reading files
100% 1/1 [03:37<00:00, 217.21s/it]
Writing ubuntu-npz/ubuntu-cleaned-4.txt.npz
Reading files
100% 1/1 [03:17<00:00, 197.16s/it]
Writing ubuntu-npz/ubuntu-cleaned-5.txt.npz
Reading files
100% 1/1 [03:26<00:00, 206.83s/it]
Writing ubuntu-npz/ubuntu-cleaned-6.txt.npz
Reading files
100% 1/1 [03:04<00:00, 184.23s/it]
Writing ubuntu-npz/ubuntu-cleaned-7.txt.npz
Reading files
100% 1/1 [02:08<00:00, 128.94s/it]
Writing ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [21]:
!du -h ubuntu-npz/*

41M	ubuntu-npz/ubuntu-cleaned-1.txt.npz
46M	ubuntu-npz/ubuntu-cleaned-2.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-3.txt.npz
41M	ubuntu-npz/ubuntu-cleaned-4.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-5.txt.npz
39M	ubuntu-npz/ubuntu-cleaned-6.txt.npz
37M	ubuntu-npz/ubuntu-cleaned-7.txt.npz
27M	ubuntu-npz/ubuntu-cleaned-8.txt.npz


In [0]:
!cp -r ubuntu-npz/ /content/drive/My\ Drive/ubuntu-npz/

In [23]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --stop_after=1501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-13 00:43:17.294896: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-13 00:43:17.295780: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e55480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-13 00:43:17.295881: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-13 00:43:17.330776: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 00:43:17.500025: I tensorflow/stream_executor/cu

In [25]:
!PYTHONPATH=src ./train.py --dataset ubuntu-npz/ --sample_every=250 --learning_rate 0.0001 --stop_after 3501 --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-10-13 02:18:45.552149: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-10-13 02:18:45.552470: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1787480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-10-13 02:18:45.552509: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-10-13 02:18:45.554721: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 02:18:45.621867: I tensorflow/stream_executor/cu

In [0]:
# clean up old saves
files = []
iterations = []
for p, d, f in os.walk('/content/gpt-2/checkpoint/'):
    for file in f:
        r = re.findall('model-(\d+).', file)
        if r:
            iterations.append(int(r[0]))
            files.append(f'{p}/{file}')


last_saved = str(max(iterations))
for f in files:
  if last_saved not in f:
    os.remove(f)

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

In [28]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.95 --model_name 345M



2019-10-13 04:06:06.270504: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:06:06.296601: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:06:06.297377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:06:06.304503: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-13 04:06:06.317636: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-13 04:06:06.323096: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [29]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=2 --top_k=100 --temperature=1



2019-10-13 04:10:26.173469: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:10:26.196431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:10:26.197379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:10:26.197806: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-13 04:10:26.199571: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-13 04:10:26.200942: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [30]:
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=2 --top_k=100 --temperature=1



2019-10-13 04:16:02.144586: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:16:02.164689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:16:02.165484: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:16:02.165823: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-13 04:16:02.167246: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-13 04:16:02.168637: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [31]:
!python3 src/interactive_conditional_samples.py  --model_name='345M'  --nsamples=10 --top_k=40 --temperature=.80 --length=1



2019-10-13 04:17:59.439217: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:17:59.459852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:17:59.460706: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:17:59.461149: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-13 04:17:59.463123: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-13 04:17:59.464950: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [32]:
#Translation
!python3 src/interactive_conditional_samples.py --model_name='345M'  --nsamples=3 --temperature=1



2019-10-13 04:20:18.008951: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-10-13 04:20:18.029017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-10-13 04:20:18.029763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2019-10-13 04:20:18.030095: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2019-10-13 04:20:18.031616: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2019-10-13 04:20:18.033329: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1